<a id="introduction"></a>
## Data Preprocessing
#### By Paul Hendricks
-------

Before we build our first machine learning model, we need to structure our data in such a way that it can fed into the model. 

In this notebook, we will also show how to preprocess data with GPU DataFrames using cuDF in RAPIDS.

**Table of Contents**

* [Data Preprocessing](#introduction)
* [Setup](#setup)
* [Mortgage Dataset](#dataset)
* [Selecting Rows or Columns](#selecting)
* [Dropping Rows or Columns](#dropping)
* [Manipulating Columns](#manipulating)
* [Transforming Columns](#transforming)
* [Renaming Columns](#renaming)
* [Modifying Data Types](#modifying)
* [Working with Missing Values](#missing)
* [Working with Indexes](#indexs)
* [Sorting Values](#sorting)
* [Merging DataFrames](#merging)
* [Concatenating DataFrames](#concatenating)
* [Aggregating with Groupbys](#groupbys)
* [One Hot Encoding](#encoding)
* [Custom Operations](#custom)
* [Conclusion](#conclusion)

<a id="setup"></a>
## Setup

This notebook was tested using the following Docker containers:

* `rapidsai/rapidsai:0.6-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub](https://hub.docker.com/r/rapidsai/rapidsai)
* `rapidsai/rapidsai-nightly:0.6-cuda10.0-devel-ubuntu18.04-gcc7-py3.7` from [DockerHub](https://hub.docker.com/r/rapidsai/rapidsai-nightly)

This notebook was run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [ ]:
!nvidia-smi

Next, let's see what CUDA version we have:

In [ ]:
!nvcc --version

<a id="dataset"></a>
## Mortgage Dataset

Going forward, we'll default to using the variable `df` to represent a Dask cuDF DataFrame.

In [ ]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import subprocess

# parse the hostname IP address
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
ip_address = str(output.decode()).split()[0]

# create a local CUDA cluster
cluster = LocalCUDACluster(ip=ip_address)
client = Client(cluster)
client

In [ ]:
%%bash

ls -alh /datasets/rapids/mortgage/mortgage_2000_1gb/perf

In [ ]:
from collections import OrderedDict
import os

base_path = os.path.join('/', 'datasets', 'rapids', 'mortgage', 'mortgage_2000_1gb')

dtypes = OrderedDict([
        ('loan_id', 'int64'),
        ('monthly_reporting_period', 'date'),
        ('servicer', 'category'),
        ('interest_rate', 'float64'),
        ('current_actual_upb', 'float64'),
        ('loan_age', 'float64'),
        ('remaining_months_to_legal_maturity', 'float64'),
        ('adj_remaining_months_to_maturity', 'float64'),
        ('maturity_date', 'date'),
        ('msa', 'float64'),
        ('current_loan_delinquency_status', 'int32'),
        ('mod_flag', 'category'),
        ('zero_balance_code', 'category'),
        ('zero_balance_effective_date', 'date'),
        ('last_paid_installment_date', 'date'),
        ('foreclosed_after', 'date'),
        ('disposition_date', 'date'),
        ('foreclosure_costs', 'float64'),
        ('prop_preservation_and_repair_costs', 'float64'),
        ('asset_recovery_costs', 'float64'),
        ('misc_holding_expenses', 'float64'),
        ('holding_taxes', 'float64'),
        ('net_sale_proceeds', 'float64'),
        ('credit_enhancement_proceeds', 'float64'),
        ('repurchase_make_whole_proceeds', 'float64'),
        ('other_foreclosure_proceeds', 'float64'),
        ('non_interest_bearing_upb', 'float64'),
        ('principal_forgiveness_upb', 'float64'),
        ('repurchase_make_whole_proceeds_flag', 'category'),
        ('foreclosure_principal_write_off_amount', 'float64'),
        ('servicing_activity_indicator', 'category')
    ])

In [ ]:
import cudf; print('cuDF Version:', cudf.__version__)
import dask_cudf; print('Dask cuDF Version:', dask_cudf.__version__)
import numpy as np; print('NumPy Version:', np.__version__)


filepath = os.path.join(base_path, 'perf', 'Performance_*')
# filepath = os.path.join(base_path, 'perf', 'Performance_2000Q1.txt_0')
df = dask_cudf.read_csv(filepath, delimiter='|', 
                        names=list(dtypes.keys()), dtype=list(dtypes.values()))

<a id="selecting"></a>
## Selecting Rows or Columns

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
print(type(df))

In [ ]:
# select rows
df_subset = df[0:4]
print(df_subset)

In [ ]:
print(type(df_subset))

In [ ]:
df_result = df_subset.compute()
print(df_result)

In [ ]:
print(type(df_result))

In [ ]:
print(df_result.shape)

In [ ]:
df.npartitions * 5

In [ ]:
# select columns
df_subset = df['loan_id']
print(df_subset)

In [ ]:
print(type(df_subset))

In [ ]:
print(df_subset.head())
print(type(df_subset.head()))

In [ ]:
df_subset = df[['loan_id', 'current_loan_delinquency_status']]
print(df_subset)

In [ ]:
print(type(df_subset))

In [ ]:
print(df_subset.head())

In [ ]:
# select both rows and columns
df_subset = df.loc[0:4, ['loan_id', 'current_loan_delinquency_status']]
print(df_subset)

In [ ]:
print(type(df_subset))

In [ ]:
df_result = df_subset.compute()
print(df_result)

In [ ]:
print(type(df_result))
print(df_result.shape)

<a id="dropping"></a>
## Dropping Rows or Columns

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
df.map_partitions(len).compute().sum()

In [ ]:
# df.drop(0:100, axis=0)

In [ ]:
df.map_partitions(len).compute().sum()

In [ ]:
df.columns

In [ ]:
# df.drop(['loan_age'], axis=1)

In [ ]:
df.columns

<a id="manipulating"></a>
## Manipulating Columns

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
df['new_column'] = df['loan_id']

In [ ]:
df.columns

In [ ]:
print(type(df))

In [ ]:
print(df['new_column'].head())

In [ ]:
# df.drop(['new_column'], axis=1)

<a id="transforming"></a>
## Transforming Columns

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="renaming"></a>
## Renaming Columns

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
df.columns

In [ ]:
# df.columns[9] = 'metropolitan_statistical_area'

In [ ]:
df.columns

In [ ]:
# df['new_column'] = df['loan_id']
# df.drop('loan_id', axis=1)

In [ ]:
df.columns

<a id="modifying"></a>
## Modifying Data Types

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
df.dtypes

In [ ]:
df.dtypes

In [ ]:
df.dtypes

<a id="missing"></a>
## Working with Missing Values

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
# calculate how many rows in each column have actual values
# # ideal
# column_counts = df.count()
# column_counts

# alternative
column_counts = []
for column in list(df.columns):
    column_count = df[column].count().compute()
    column_counts.append((column, column_count))

In [ ]:
number_of_rows = df.map_partitions(len).compute().sum()

In [ ]:
for column, count in column_counts:
    print(column, ':', (count / number_of_rows) * 100)

In [ ]:
# # loop over each column in the dataframe and each column's dtype
# for column, data_type in df.dtypes.items():
#     # if the data type is not numeric, cast to int32 and fill with -1
#     if str(data_type) == "category":
#         df[column] = df[column].astype('int32').fillna(-1)

#     # if the data type is numeric, cast to appropriate type and fill with -1
#     if str(data_type) in ['int8', 'int16', 'int32', 'int64', 'float32', 'float64']:
#         df[column] = df[column].fillna(np.dtype(data_type).type(-1))

In [ ]:
# df.persist()

In [ ]:
# calculate how many rows in each column have actual values
# # ideal
# column_counts = df.count()
# column_counts

# alternative
column_counts = []
for column in list(df.columns):
    column_count = df[column].count().compute()
    column_counts.append((column, column_count))

In [ ]:
for column, count in column_counts:
    print(column, ':', (count / number_of_rows) * 100)

<a id="indexes"></a>
## Working with Indexes

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="sorting"></a>
## Sorting Values

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="merging"></a>
## Merging DataFrames

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
!ls -alh /datasets/rapids/mortgage/mortgage_2000_1gb/acq

In [ ]:
dtypes = OrderedDict([
        ('loan_id', 'int64'),
        ('orig_channel', 'category'),
        ('seller_name', 'category'),
        ('orig_interest_rate', 'float64'),
        ('orig_upb', 'int64'),
        ('orig_loan_term', 'int64'),
        ('orig_date', 'date'),
        ('first_pay_date', 'date'),
        ('orig_ltv', 'float64'),
        ('orig_cltv', 'float64'),
        ('num_borrowers', 'float64'),
        ('dti', 'float64'),
        ('borrower_credit_score', 'float64'),
        ('first_home_buyer', 'category'),
        ('loan_purpose', 'category'),
        ('property_type', 'category'),
        ('num_units', 'int64'),
        ('occupancy_status', 'category'),
        ('property_state', 'category'),
        ('zip', 'int64'),
        ('mortgage_insurance_percent', 'float64'),
        ('product_type', 'category'),
        ('coborrow_credit_score', 'float64'),
        ('mortgage_insurance_type', 'float64'),
        ('relocation_mortgage_indicator', 'category')
    ])

In [ ]:
filepath = os.path.join(base_path, 'acq', 'Acquisition_2000Q1.txt')
# filepath = os.path.join(base_path, 'perf', 'Acquisition_')
acq_df = dask_cudf.read_csv(filepath, delimiter='|', 
                            names=list(dtypes.keys()), dtype=list(dtypes.values()))

In [ ]:
print(acq_df.head())

In [ ]:
# calculate number of rows
acq_df.map_partitions(len).compute().sum()

In [ ]:
acq_df.dtypes

In [ ]:
# acq_df['orig_date'] = acq_df['orig_date'].astype(np.datetime64)
# acq_df['first_pay_date'] = acq_df['first_pay_date'].astype(np.datetime64)

In [ ]:
acq_df.dtypes

In [ ]:
# acq_df.drop(['orig_date', 'first_pay_date'], axis=1)

In [ ]:
acq_df.head()

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
!head /datasets/rapids/mortgage/mortgage_2000_1gb/names.csv

In [ ]:
dtypes = OrderedDict([
        ("seller_name", "category"),
        ("new", "category"),
    ])

In [ ]:
filepath = os.path.join(base_path, 'names.csv')
# filepath = os.path.join(base_path, 'perf', 'Performance_2000Q1.txt_0')
names_df = dask_cudf.read_csv(filepath, delimiter='|', 
                              names=list(dtypes.keys()), dtype=list(dtypes.values()))

In [ ]:
print(names_df.head())

In [ ]:
names_df.dtypes

In [ ]:
# calculate number of rows
names_df.map_partitions(len).compute().sum()

In [ ]:
print(names_df.head())

In [ ]:
print(acq_df.head())

In [ ]:
print(type(acq_df), type(names_df))

In [ ]:
acq_df.dtypes

In [ ]:
# acq_df.drop(['orig_date', 'first_pay_date'], axis=1)
subset_columns = [i for i in acq_df.columns if i not in ['orig_date', 'first_pay_date']]
print(subset_columns)

In [ ]:
merged_df = acq_df[subset_columns].merge(names_df, how='left', on=['seller_name'])

In [ ]:
acq_df.dtypes

In [ ]:
merged_df.dtypes

In [ ]:
# calculate number of rows
merged_df.head()

<a id="concatenating"></a>
## Concatenating DataFrames

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

In [ ]:
df_delayed = df.to_delayed()
df_delayed

In [ ]:
# [(gpu_df, list(client.who_has(gpu_df).values())[0]) for gpu_df in gpu_dfs]

# partition_worker_map = [(partition, list(client.who_has(partition).values())[0]) for partition in df]
# [client.who_has(partition) for partition in df ]

In [ ]:
from dask.delayed import delayed


def head(dataframe):
    return dataframe.head()


dfs = [delayed(head)(d) for d in df_delayed]

In [ ]:
from dask.distributed import wait

futures = client.compute(dfs)
wait(futures)
futures

In [ ]:
# results = [result.result() for future in futures]
results = client.gather(futures)

In [ ]:
result_worker_map = {result: list(client.who_has(result).values())[0] for result in results}
import time; time.sleep(3)
result_worker_map

<a id="groupbys"></a>
## Aggregating with Groupbys

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="encoding"></a>
## One Hot Encoding

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="custom"></a>
## Custom Operations

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

<a id="conclusion"></a>
## Conclusion

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)